In [5]:
# Bibliotecas padrão
import os
import time
import logging
from typing import Dict, Tuple, Optional, List, Any
from dataclasses import dataclass
from enum import Enum
from datetime import datetime

# Bibliotecas de terceiros
import numpy as np
import pandas as pd
import requests
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Configuração do logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [6]:
@dataclass
class CoeficienteANTT:
    """Coeficientes de custo conforme tabela ANTT"""
    deslocamento: float  # CCD - R$/km
    carga_descarga: float  # CC - R$


def criar_tabela_antt():
    """
    Cria uma tabela ANTT estruturada para fácil leitura
    """
    # Estrutura da tabela
    dados = {
        'tipo_carga': [
            'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido', 'Granel sólido',
            'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido', 'Granel líquido',
            'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada', 'Frigorificada',
            'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada', 'Conteinerizada',
            'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral', 'Carga Geral'
        ],
        'num_eixos': [
            2, 3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9,
            3, 4, 5, 6, 7, 9,
            2, 3, 4, 5, 6, 7, 9
        ],
        'coef_deslocamento': [  # CCD - R$/km
            3.4880, 4.4505, 5.1405, 5.5749, 6.2063, 7.0506, 7.9437,
            3.5634, 4.5518, 5.1213, 5.7255, 6.4428, 7.1940, 8.2354,
            4.1977, 5.3292, 6.1395, 6.7886, 7.4860, 8.9162, 9.8118,
            4.4235, 5.0137, 5.5159, 6.1318, 7.0371, 7.8991,
            3.4645, 4.4360, 5.0845, 5.5455, 6.1249, 7.0577, 7.9940
        ],
        'coef_carga_descarga': [  # CC - R$
            408.57, 495.60, 536.30, 520.16, 546.66, 710.36, 755.95,
            423.65, 517.81, 520.16, 550.70, 600.82, 738.92, 825.27,
            490.43, 589.50, 640.20, 658.43, 675.05, 999.88, 1011.88,
            488.19, 501.44, 503.95, 526.18, 706.64, 743.67,
            402.10, 491.61, 520.92, 512.07, 524.27, 712.31, 769.78
        ]
    }
    
    # Cria o DataFrame
    df = pd.DataFrame(dados)
    
    # Salva em Excel
    df.to_excel('tabela_antt_estruturada.xlsx', index=False)
    print("Tabela ANTT estruturada criada com sucesso!")
    
    return df

def processar_tabela_antt(df: pd.DataFrame) -> Dict[str, Dict[int, CoeficienteANTT]]:
    """
    Processa a tabela ANTT estruturada e retorna um dicionário com os coeficientes
    """
    coeficientes = {}
    
    # Processa cada linha da tabela
    for _, row in df.iterrows():
        tipo_carga = row['tipo_carga']
        num_eixos = int(row['num_eixos'])
        
        if tipo_carga not in coeficientes:
            coeficientes[tipo_carga] = {}
            
        coeficientes[tipo_carga][num_eixos] = CoeficienteANTT(
            deslocamento=float(row['coef_deslocamento']),
            carga_descarga=float(row['coef_carga_descarga'])
        )
    
    # Validação e log
    print("\nTipos de carga processados:")
    for tipo_carga, configs in coeficientes.items():
        print(f"{tipo_carga}: {len(configs)} configurações de eixos")
        
    return coeficientes



class CalculadoraDistancia:
    def __init__(self):
        """Inicializa a calculadora com OSRM"""
        self.osrm_url = "http://router.project-osrm.org/route/v1/driving"
        self.geocoder = Nominatim(user_agent="calculadora_frete")
    
    def obter_coordenadas(self, cidade: str) -> Optional[Tuple[float, float]]:
        """
        Obtém as coordenadas de uma cidade usando Nominatim
        Args:
            cidade: Nome da cidade
        Returns:
            Tupla com (longitude, latitude) ou None se não encontrar
        """
        try:
            location = self.geocoder.geocode(f"{cidade}, Brasil")
            time.sleep(1)  # Respeita limites da API
            
            if location:
                return (location.longitude, location.latitude)
            return None
            
        except Exception as e:
            print(f"Erro ao obter coordenadas de {cidade}: {str(e)}")
            return None
    
    def calcular_distancia_rota(self, origem: str, destino: str) -> Optional[Dict]:
        """
        Calcula a distância real da rota entre duas cidades usando OSRM
        Args:
            origem: Cidade de origem
            destino: Cidade de destino
        Returns:
            Dicionário com distância em km e duração em horas
        """
        try:
            # Obtém coordenadas
            coords_origem = self.obter_coordenadas(origem)
            coords_destino = self.obter_coordenadas(destino)
            
            if not coords_origem or not coords_destino:
                raise ValueError("Não foi possível obter coordenadas")
            
            # Monta URL da requisição
            coords = f"{coords_origem[0]},{coords_origem[1]};"
            coords += f"{coords_destino[0]},{coords_destino[1]}"
            url = f"{self.osrm_url}/{coords}"
            
            # Faz a requisição
            response = requests.get(url)
            if response.status_code != 200:
                raise ValueError("Erro ao obter rota do OSRM")
            
            data = response.json()
            if data["code"] != "Ok":
                raise ValueError("Rota não encontrada")
            
            # Extrai informações
            route = data["routes"][0]
            distancia_km = round(route["distance"] / 1000, 2)
            duracao_horas = round(route["duration"] / 3600, 2)
            
            return {
                'distancia_km': distancia_km,
                'duracao_horas': duracao_horas,
                'coordenadas': {
                    'origem': coords_origem,
                    'destino': coords_destino
                }
            }
            
        except Exception as e:
            print(f"Erro ao calcular rota: {str(e)}")
            return None

class CalculadoraFreteANTT:
    def __init__(self, tabela_antt: pd.DataFrame) -> None:
        """Inicializa a calculadora com a tabela ANTT"""
        self.logger = logging.getLogger(__name__)
        self.calculadora_distancia = CalculadoraDistancia()
        self.coeficientes = processar_tabela_antt(tabela_antt)
    
    def calcular_frete(self, 
                      origem: str, 
                      destino: str, 
                      tipo_carga: str,
                      num_eixos: int) -> Optional[Dict]:
        """
        Calcula o frete usando os coeficientes ANTT e distância real
        Frete = CC + (CCD * distância)
        """
        try:
            # Verifica coeficientes
            if tipo_carga not in self.coeficientes or num_eixos not in self.coeficientes[tipo_carga]:
                raise ValueError(f"Combinação inválida: {tipo_carga} com {num_eixos} eixos")

            # Calcula distância real
            rota = self.calculadora_distancia.calcular_distancia_rota(origem, destino)
            if not rota:
                raise ValueError("Não foi possível calcular a distância")

            # Obtém coeficientes
            coef = self.coeficientes[tipo_carga][num_eixos]

            # Calcula frete
            valor_frete = coef.carga_descarga + (coef.deslocamento * rota['distancia_km'])

            return {
                "origem": origem,
                "destino": destino,
                "tipo_carga": tipo_carga,
                "distancia_km": rota['distancia_km'],
                "duracao_horas": rota['duracao_horas'],
                "num_eixos": num_eixos,
                "coeficientes": {
                    "deslocamento": coef.deslocamento,
                    "carga_descarga": coef.carga_descarga
                },
                "valor_frete": round(valor_frete, 2)
            }

        except Exception as e:
            self.logger.error(f"Erro no cálculo: {str(e)}")
            return None
        
def criar_dados_entrada():
    """
    Cria arquivo de dados de entrada para teste
    Returns:
        pd.DataFrame: DataFrame com os dados de entrada
    """
    # Cria dados de teste
    dados_teste = {
        'origem': ['São Paulo', 'Curitiba', 'Porto Alegre'],
        'destino': ['Rio de Janeiro', 'Salvador', 'Manaus'],
        'tipo_carga': ['Granel sólido', 'Granel líquido', 'Carga Geral'],
        'num_eixos': [3, 4, 5]
    }
    
    # Cria DataFrame
    df_entrada = pd.DataFrame(dados_teste)
    
    # Salva arquivo
    df_entrada.to_excel('dados_entrada.xlsx', index=False)
    print("\nArquivo de entrada criado com sucesso!")
    
    return df_entrada

def exportar_excel(df_entrada: pd.DataFrame, calculadora: CalculadoraFreteANTT) -> None:
    """
    Exporta os resultados dos cálculos para Excel
    Args:
        df_entrada: DataFrame com os dados de entrada
        calculadora: Instância da calculadora de frete
    """
    resultados = []
    
    for _, row in df_entrada.iterrows():
        resultado = calculadora.calcular_frete(
            origem=row['origem'],
            destino=row['destino'],
            tipo_carga=row['tipo_carga'],
            num_eixos=row['num_eixos']
        )
        if resultado:
            # Separa os coeficientes em campos individuais
            resultado_formatado = {
                'origem': resultado['origem'],
                'destino': resultado['destino'],
                'tipo_carga': resultado['tipo_carga'],
                'distancia_km': resultado['distancia_km'],
                'duracao_horas': resultado['duracao_horas'],
                'num_eixos': resultado['num_eixos'],
                'coef_deslocamento': resultado['coeficientes']['deslocamento'],
                'coef_carga_descarga': resultado['coeficientes']['carga_descarga'],
                'valor_frete': resultado['valor_frete']
            }
            print(f"\nResultado para {row['origem']} -> {row['destino']}:")
            print(f"Distância: {resultado['distancia_km']} km")
            print(f"Duração: {resultado['duracao_horas']} horas")
            print(f"Valor do frete: R$ {resultado['valor_frete']}")
            
            resultados.append(resultado_formatado)
        else:
            resultados.append({
                'erro': 'Falha no cálculo',
                **row.to_dict()
            })
    
    # Salva resultados
    df_resultados = pd.DataFrame(resultados)
    
    # Reordena as colunas
    colunas_ordem = [
        'origem', 'destino', 'tipo_carga', 'distancia_km', 'duracao_horas', 
        'num_eixos', 'coef_deslocamento', 'coef_carga_descarga', 'valor_frete'
    ]
    df_resultados = df_resultados[colunas_ordem]
    
    df_resultados.to_excel('resultados.xlsx', index=False)
    print("\nProcessamento concluído com sucesso!")

def main():
    """Função principal"""
    try:
        # Cria e lê a tabela ANTT estruturada
        tabela_antt = criar_tabela_antt()
        
        # Cria dados de entrada
        df_entrada = criar_dados_entrada()
        
        # Processa cálculos
        calculadora = CalculadoraFreteANTT(tabela_antt)
        
        # Exporta resultados
        exportar_excel(df_entrada, calculadora)
        
    except Exception as e:
        print(f"Erro durante a execução: {str(e)}")

if __name__ == "__main__":
    main()

Tabela ANTT estruturada criada com sucesso!

Arquivo de entrada criado com sucesso!

Tipos de carga processados:
Granel sólido: 7 configurações de eixos
Granel líquido: 7 configurações de eixos
Frigorificada: 7 configurações de eixos
Conteinerizada: 6 configurações de eixos
Carga Geral: 7 configurações de eixos

Resultado para São Paulo -> Rio de Janeiro:
Distância: 442.77 km
Duração: 5.67 horas
Valor do frete: R$ 2466.15

Resultado para Curitiba -> Salvador:
Distância: 2358.98 km
Duração: 30.87 horas
Valor do frete: R$ 12601.2

Resultado para Porto Alegre -> Manaus:
Distância: 4329.69 km
Duração: 67.32 horas
Valor do frete: R$ 24522.37

Processamento concluído com sucesso!
